## Set Exclcusions

In [1]:
$exclude='tax|pensions|life|retire|twitter'

## Load Get-SQL module, copy browser history, connect to it. 
The chromium engine puts data in 
```
<<userProfile>>
    \AppData
        \Local
            \Publisher          {Google | Microsoft}
                \BrowserName    {Chrome | Edge | Edge Dev | etc}
                    \User Data
                        \Default
```
The first file to look at is **History**

In [1]:
cd ~
$browserDir = '.\AppData\Local\Microsoft\Edge Dev\User Data\Default'
if (-not (Test-Path -PathType Container ".\temp")) {mkdir '.\temp'}

Import-Module getsql -PassThru | Format-Table name,version  | Out-String #Out-string is better in beta notebooks

Copy-Item "$browserDir\History" -Destination '.\temp'
$tables = Get-SQL  -Connection .\temp\history  -Lite  -ShowTables
$tables -join '; '


Name   Version
----   -------
getsql 1.3.0.4




content_annotations; downloads; downloads_slices; downloads_url_chains; keyword_search_terms; meta; segment_usage; segments; sqlite_sequence; typed_url_sync_metadata; urls; visit_source; visits


In [1]:
# use SQL as an alias, Examine the table 'visits'
sql -Describe 'visits' | Out-String


COLUMN_NAME                     TYPE_NAME COLUMN_SIZE IS_NULLABLE
-----------                     --------- ----------- -----------
id                                                          False
url                                                         False
visit_time                                                  False
from_visit                                                   True
transition                                                  False
segment_id                                                   True
visit_duration                                              False
incremented_omnibox_typed_score                             False
publicly_routable                                           False




## Oldest downloads in my history - from when I first got the machine.

In [1]:
# Dates are stored as microseconds since 1st Jan 1601; 
# So: convert to seconds, subtract time since '1601-01-01' (could optimize as -11644473600 but less readable)
# and convert to a string representing date.
$Downloads = sql @'
    SELECT   datetime(   start_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS StartTime,
             datetime(   end_time   / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS EndTime,
            ((end_time - start_time)/ 1000000 )               AS Duration, 
            (received_bytes*1000000 /(end_time - start_time)) AS BytesPerSec,
             received_bytes, state, tab_url, target_path, mime_type
    FROM     downloads
    ORDER BY start_time 
    LIMIT    25
'@ 

$Downloads | where tab_Url -notmatch $exclude | Format-Table -Wrap -AutoSize -property StartTime, 
                        @{n='Total KB'; e={$_.received_bytes/1kb};         a='Right';f='#,###' }, 
                        @{n='KB/Sec'  ; e={($_.BytesPerSec/1kb)}; a='right';f='0'; }, 
                        @{n='URL';e='tab_url'}  | Out-String -Width 190  #Out-String is just for beta notebooks

25 row(s) returned



StartTime           Total KB KB/Sec URL
---------           -------- ------ ---
2019-04-16 08:50:56       39      9 https://github.com/PowerShell/PowerShell-RFC/blob/master/CommunityCall/PSTeamsCall.ics
2019-04-19 16:07:06      125      4 https://cki3.checkin.lufthansa.com/boardingPasses/LH_WEBCKI.DE.STANDALONE.0LkglLFXPqMsAbR82MXW8-.pdf
2019-04-25 16:58:42      921     72 https://www.ncsc.gov.uk/blog-post/passwords-passwords-everywhere
2019-04-25 18:19:42      390    335 https://docs.microsoft.com/en-us/microsoftteams/teams-architecture-solutions-posters
2019-04-25 18:19:57      187    411 https://docs.microsoft.com/en-us/microsoftteams/teams-architecture-solutions-posters
2019-04-25 18:20:07      251    287 https://docs.microsoft.com/en-us/microsoftteams/teams-architecture-solutions-posters
2019-04-25 18:21:04      327    269 https://docs.microsoft.com/en-us/microsoftteams/teams-architecture-solutions-posters
2019-04-25 18:22:53      401    311 https://docs.microsoft.com/en-us/micro

## Get Search terms from history

In [1]:
# This time build the query from the command line -table <<Tab to name>> -select <<Tab to column>> -distinct  "extra SQL"
sql -Table "keyword_search_terms" -Select "term" -Distinct "Limit 20"  | where term -notmatch $exclude | out-string -width 190

$moreComplexQuery =  @'
    SELECT  term,
            urls.url,
            datetime(urls.last_visit_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS visitTime
    FROM keyword_search_terms
    JOIN urls on keyword_search_terms.url_id = urls.id
'@

20 row(s) returned



term
----
"/v1/lens/focus"
"1% NHS"
"Adrian Dunbar"
"Advise like youth probably wasted on the young"
"All 12"
"Amy Winehouse"
"Andy Burnham"
"Ant Middleton"
"Aston Martin"
"BBC TV"
"Barack Obama"
"Barry Cryer"
"Beth Rigby"
"Big Mac"
"Billy Connolly"
"Channel 5"
"Charles Kennedy"
"Chris Whitty"
"Cliff Richard"
"Daniel Craig"




## Most visited sites and most recently visited from History

In [1]:
sql -Describe "visits" | Out-String #out-string for notebook


COLUMN_NAME                     TYPE_NAME COLUMN_SIZE IS_NULLABLE
-----------                     --------- ----------- -----------
id                                                          False
url                                                         False
visit_time                                                  False
from_visit                                                   True
transition                                                  False
segment_id                                                   True
visit_duration                                              False
incremented_omnibox_typed_score                             False
publicly_routable                                           False




In [1]:
$Grouped = sql @"
    SELECT  count (visit_Time) as visits , 
            urls.url
    FROM  visits JOIN  urls
      ON  visits.url = urls.id
    WHERE    (visits.visit_time < 100000000000000000) 
    Group BY urls.url
    Order by visits desc
"@  | where url -NotMatch $exclude  | Group-Object -Property @{e={$_.url -replace "(?<!/)/(?!/).*$",""}}

$totals = foreach ($g in $grouped) {$g.group | measure -sum visits | select @{n='name'; e={$g.name}} , Sum}
$totals | sort sum -Descending | select -first 20


In [1]:
SQL "SELECT * FROM segments join segment_usage on segment_usage.segment_id = segments.id join urls on urls.id = segments.url_id limit 5" |Out-String -Width 190

In [1]:
# close the SQL connection after this one. Give it few seconds before deleting the file from \temp - in the next block 
sql @"
    SELECT  datetime(visit_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS visitTime , 
            urls.url,  urls.title
    FROM  visits JOIN  urls
      ON  visits.url = urls.id
    WHERE    (visit_time < 100000000000000000) 
    ORDER BY visit_time DESC
    LIMIT 100
"@  | where url -NotMatch $exclude | Select -first 20 |  Out-String -Width 190

sql -close 

In [1]:
cd ~
del  '.\temp\history' #-ErrorAction SilentlyContinue
copy "$browserDir\Top Sites" -Destination '.\temp'
sql -ForceNew -Lite -Connection '.\temp\top sites'-Table "top_sites" -OrderBy "url_rank" -where "url_rank" -lt 10 -close | where url -NotMatch $exclude | Out-String

## Collections in Microsft Edge (ChrEdge from versions ~80 onwards)

In [1]:
cd ~
$browserDir = '.\AppData\Local\Microsoft\Edge Dev\User Data\Default'
if (-not (Test-Path -PathType Container ".\temp")) {mkdir '.\temp'}

Import-Module getsql

del  '.\temp\top sites' -ErrorAction SilentlyContinue

copy "$browserDir\Collections\collectionsSQLite" -Destination '.\temp'
Get-SQL -Lite -Connection '.\temp\collectionsSQLite' -forcenew -ShowTables |Out-String
# close the SQL connection after this one. Give it few seconds before deleting the file from .\temp - in the next block 
$collectionrows = Sql @'
    SELECT
        collections_items_relationship.position,
        collections.title AS Collection,
        items.title,
        items.source,
        items.text_content
    FROM                            collections_items_relationship
    JOIN      items       on        collections_items_relationship.item_id   = items.id  
    JOIN      collections on        collections_items_relationship.parent_id = collections.id 
    ORDER BY  collections.position, collections_items_relationship.position
'@ -Close | Select-Object -Property Collection,Position, Title,
                @{n='LinkSource';   e={(ConvertFrom-Json ([string]::new($_.source))).url} },
                Text_Content, 
                @{n='LinkSiteName'; e={(ConvertFrom-Json ([string]::new($_.source))).websiteName} }

collections
collections_items_relationship
collections_sync
comments
favicons
items
items_offline_data
items_sync
meta



213 row(s) returned


In [1]:
$collectionrows | Format-Table -AutoSize -wrap | Out-String -Width 190

96 row(s) returned



Collection     position title
----------     -------- -----                                                                                                                                                                 
Training Pile         1 How I accidentally created a creepy Halloween bot ... and you can too! - DEV
Training Pile         3 Azure Functions: "Azure Functions - live via Restream.io"
Training Pile         4 Writing an Extension Vault for PowerShell SecretManagement Preview 4
Training Pile         5 Azure Web App Service and GitHub Actions (Video Tutorial) | Azure DevOps Blog
Training Pile         6 How to make your Azure DevOps CI/CD pipeline faster - Tim Deschryver
Training Pile         7 An API layer of PowerShell-based Function Apps - a few tips
Training Pile         8 Remote State Management with Terraform Cloud - YouTube
Training Pile         9 Using Terraform with Azure | Microsoft Docs
Training Pile        10 Create a Kubernetes cluster with Azure Kubernetes Se

In [1]:
#alternatively   
# Find duplicates 
#$collectionrows | group -Property LinkSource | where count -ge 2 | select -ExpandProperty group 

$collectionrows | Export-Excel
del  '.\temp\collectionsSQLite' -ErrorAction SilentlyContinue

## Bookmarks are a JSON file. Let's Parse that :-) 

In [1]:
del  '.\temp\collectionsSQLite' -ErrorAction SilentlyContinue
 
$b     = ConvertFrom-Json -AsHashtable -InputObject (Get-Content "$browserDir\Bookmarks" -raw) -Depth 10
$links = $(foreach ($k in $b.roots.Keys) {foreach ($c in $b.roots[$k].children) {[pscustomObject]$c}} ) 
$links | where url -notmatch $exclude | Format-Table @{w=50;e='name'},@{w=50;e='url'} -Wrap 

name                                               url


----                                               ---


Tips on how to take Microsoft Azure Certification  https://www.thomasmaurer.ch/2019/09/tips-on-how-


Exams - Thomas Maurer                              to-take-microsoft-azure-certification-exams/


My Visual Studio                                   https://my.visualstudio.com/Benefits?wt.mc_id=o~


...

Registry                                           ork-security-group/azurerm/3.0.1?tab=inputs


Special Offers | Bargain & Discount Gifts -        https://www.virginexperiencedays.co.uk/special-o


Virgin Experience Days                             ffers 


What to do when your build hangs on the Hosted     https://jessehouwing.net/what-to-do-when-your-bu


Pool...                                            ild-hangs-on-the-hosted-pool/


## Things for another time ... 

In [1]:
<#
#Downloads
sql -Table "autofill" -OrderBy "count" | select  name,count,@{n='value';e={Unprotect  $_.value}} | export-excel

get-sql -Lite -Connection 'C:\temp\Web Data'
sql -Describe "credit_cards"    
COLUMN_NAME           TYPE_NAME COLUMN_SIZE IS_NULLABLE
-----------           --------- ----------- -----------
guid                                              False
name_on_card                                       True
expiration_month                                   True
expiration_year                                    True
card_number_encrypted                              True
date_modified                                     False
origin                                             True
use_count                                         False
use_date                                          False
billing_address_id                                 True
nickname                                           True

get-sql -Lite -Connection 'C:\temp\History' -ShowTables -ForceNew
SQL "SELECT visits.id, visits.visit_time,visits.visit_duration, urls.url, urls.title FROM visits join urls limit 10" 

SQL "SELECT * FROM urls order by visit_count desc limit 10" | ft visit_count,url

SQL "SELECT * FROM segments join segment_usage on segment_usage.segment_id = segments.id join urls on urls.id = segments.url_id limit 5"


sql -lite -connection C:\temp\History -ForceNew -sql  @"
    SELECT  visit_count, typed_count, title,
            datetime(last_visit_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS lastVisitTime,
            url
    FROM urls
    ORDER BY visit_count DESC
"@  -close  | where url -NotMatch "job|mail|purple" | ogv -Title "My top urls"



sql -lite -connection C:\temp\History -ForceNew -sql  @"
    SELECT  datetime(visit_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS visitTime,
            urls.url, urls.title, visit_duration
    FROM visits
    JOIN urls on visits.url = urls.id
    ORDER BY visit_time DESC
    limit 100
"@  -close -GridView

sql -lite -connection C:\temp\History -ForceNew -sql  @"
    SELECT  term,
            urls.url
    FROM keyword_search_terms
    JOIN urls on keyword_search_terms.url_id = urls.id

"@ -close | where url -notmatch "wives|revelations|tax|pensions|life|retire" | ogv -Title "keyword search terms"

sql -Lite -Connection "C:\temp\Cookies" -sql "Select host_key, name,path, has_expires, datetime(expires_utc / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS expires, datetime(last_access_utc / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch') AS lastUse, samesite,source_scheme from cookies  limit 50" -grid -close 

sql -Lite -Connection "C:\temp\Network Action Predictor" -Table "network_action_predictor" -grid -Close

#top sites - most visited

Sql @"
    select
        collections_items_relationship.position,
        collections.title,
        items.title,
        items.source,
        items.text_content
    from      collections_items_relationship
        join  items       on items.id       = collections_items_relationship.item_id
        Join  collections on collections.id = collections_items_relationship.parent_id
    order by  collections.position,           collections_items_relationship.position
"@ | ogv

$b = ConvertFrom-Json -AsHashtable -InputObject (Get-Content C:\temp\Bookmarks -raw) -Depth 10
$links =  foreach ($c in  $b.roots.Values.children) {New-Object psobject -Property $c }
$links | ft name,url -AutoSize -Wrap

get-sql -Connection "Driver={SQLite3 ODBC Driver};Database=C:\temp\Collections\collectionsSQLite"  -forcenew
Sql @"
     select
         collections_items_relationship.position,
         collections.title as Collection,
         items.title,
         items.source,
         items.text_content
     from      collections_items_relationship
         join  items       on items.id       = collections_items_relationship.item_id
         Join  collections on collections.id = collections_items_relationship.parent_id
     order by  collections.position           , collections_items_relationship.position
"@ | select -Property Position,Collection,Title, Text_Content, @{n='LinkSource'; e={
    (ConvertFrom-Json (($_.source -replace "^x'", '' -replace "'",'' -replace  "(\S\S)",'$1 ').trim().split(" ").ForEach({[char][System.Convert]::ToInt16($_,16)}) -join "")).url} }

Web Data      autofill incl credit card info
#  https://www.dataforensics.org/retrieve-saved-password-in-google-chrome/
#>
#..\safe browsing cookies
#..\adblocking\blocklist json file
#..\Functional Data